In [1]:
import sys
sys.path.append('../../build')
sys.path.append('../')
import IPSModule as ips
import numpy as np
import matplotlib.pyplot as plt

import numpy as np
import lattpy as lp

from utils import *

In [2]:
def verify_kinetic_energy_simulation(gamma=1.0, temperature=0.3, rad=10.0,
                                     epsilon=1.0, sigma=1.0,
                                     steps_per_iter=100, num_iter=500,
                                     dt=0.001, show_configuration=False):
    # generate initial configuration
    init_dis = 2 ** (1.0 / 6.0) * sigma
    np.random.seed(42) 
    # print("Initial spacing:", init_dis)

    # generate a hexagonal lattice
    _dis = init_dis * 2 / np.sqrt(3)
    latt = lp.Lattice.hexagonal(a=_dis)
    latt.add_atom()
    latt.add_connections()
    
    # build a donut shape configuration (inner radius 3, outer radius 6)
    s = lp.Donut((0, 0), radius_outer=6, radius_inner=3)
    latt.build(shape=s, primitive=True)

    if show_configuration:
        ax = latt.plot()
        s.plot(ax)
        plt.title("Initial Configuration")
        plt.show()

    num_particles = latt.data.positions.shape[0]
    init_particles_positions = latt.data.positions

    # initialize the system
    p = ips.LangevinSystem(num_particles, gamma, temperature)
    for i in range(num_particles):
        for d in range(2):
            p.get_positions()[d][i] = init_particles_positions[i][d] / 2
            p.get_velocities()[d][i] = 0.0

    # setting up the simulator
    pair_force_config = {
        "type": "LennardJones",
        "eps": epsilon,
        "sigma": sigma
    }
    confinement_config = {
        "type": "Radial",
        "rad": rad
    }
    simulator = ips.IPS_Simulator_Langevin(p)
    simulator.init(pair_force_config, confinement_config)

    # run the simulation and record kinetic energy
    kinetic_energy_list = []
    time_steps = []
    total_steps = 0

    for it in range(num_iter):
        ke = calculate_total_kinetic_energy(p.get_velocities())
        kinetic_energy_list.append(ke)
        time_steps.append(total_steps)
        simulator.integrate_n_steps(dt, steps_per_iter)
        total_steps += steps_per_iter

    # Theoretical expected energy (in 2D, each particle's kinetic energy should be T if mass = 1 and k_B = 1)
    expected_energy = num_particles * temperature

    # plot kinetic energy evolution vs simulation steps
    plt.figure(figsize=(10, 6))
    plt.plot(time_steps, kinetic_energy_list, 'b.-', label="Simulated Kinetic Energy")
    plt.axhline(expected_energy, color='r', linestyle='--', 
                label=f"Theoretical Energy = {expected_energy:.3f}")
    plt.xlabel("Simulation Steps")
    plt.ylabel("Total Kinetic Energy")
    plt.title(f"Kinetic Energy Evolution (Temperature = {temperature}, Gamma = {gamma})")
    plt.legend()
    plt.grid(True)
    plt.show()

    mean_energy = np.mean(kinetic_energy_list[:-200])
    print(f"Mean Kinetic Energy: {mean_energy:.3f}")
    print(f"Expected Kinetic Energy: {expected_energy:.3f}")
    

    return time_steps, kinetic_energy_list, expected_energy

In [ ]:
time_steps, kinetic_energy_list, expected_energy = verify_kinetic_energy_simulation()

In [ ]:
time_steps, kinetic_energy_list, expected_energy = verify_kinetic_energy_simulation(gamma=10)

In [ ]:
time_steps, kinetic_energy_list, expected_energy = verify_kinetic_energy_simulation(gamma=0.1)